# Setup

In [195]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

!pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels-test/repo.html


In [196]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn

# My Graph

In [197]:
g = dgl.heterograph({
    ('ID', 'follower', 'ID'): (torch.tensor([0, 1, 1, 2]), torch.tensor([1, 2, 3, 0])),
    ('ID', 'Cinema', 'Video'): (torch.tensor([3, 1, 2]), torch.tensor([5, 4, 6])),
    ('Video', 'belongs_to', 'genre'): (torch.tensor([4, 5, 6]), torch.tensor([7, 8, 9]))
})


In [198]:
f = dgl.graph((torch.tensor([1, 1, 0, 3]), torch.tensor([2, 3, 3, 1])))

In [199]:
b = dgl.heterograph({
    ('ID', 'follower', 'name'): (torch.tensor([0]), torch.tensor([1])),
    ('Video', 'belongs_to', 'genre'): (torch.tensor([1]), torch.tensor([2]))
})

In [200]:
n =  dgl.heterograph({
    ('ID', 'follower', 'ID'): (torch.tensor([0]), torch.tensor([1])),
    ('Video', 'belongs_to', 'genre'): (torch.tensor([1]), torch.tensor([2]))
})

# Querying graph structure

Return the number of nodes in the graph

In [201]:
g.num_nodes('ID')

4

In [202]:
g.num_nodes('Video')

7

Alias of num_nodes()

In [203]:
g.num_nodes()

21

Return the number of edges in the graph

In [204]:
g.num_edges('follower')

4

In [205]:
g.num_edges('Cinema')

3

In [206]:
g.num_edges('belongs_to')

3

In [207]:
g.num_edges(('ID', 'follower', 'ID'))

4

In [208]:
g.num_edges(('ID', 'Cinema', 'Video'))

3

In [209]:
g.num_edges(('Video', 'belongs_to', 'genre'))

3

 Alias of num_edges()



In [210]:
g.num_edges()

10

Return the number of source nodes in the graph

In [211]:
g.num_src_nodes('ID')

4

In [212]:
g.num_src_nodes('Video')

7

In [213]:
g.num_src_nodes('genre')

10

 Alias of num_src_nodes()

In [214]:
g.num_src_nodes()

21

 Return the number of destination nodes in the graph.

In [215]:
g.num_dst_nodes('ID')

4

In [216]:
g.num_dst_nodes('Video')

7

In [217]:
g.num_dst_nodes('genre')

10

 Alias of num_dst_nodes()

In [218]:
g.num_dst_nodes()

21

 Return whether the graph is a uni-bipartite graph.

In [219]:
g.is_unibipartite

False

 Return whether the graph is a multigraph with parallel edges

In [220]:
g.is_multigraph

False

Return whether the graph is a homogeneous graph.



In [221]:
g.is_homogeneous

False

Return whether the graph contains the given nodes.



In [222]:
g.has_nodes(0, 'ID')

True

In [223]:
g.has_nodes(1, 'ID')

True

In [224]:
g.has_nodes(4, 'ID')

False

In [225]:
g.has_nodes(4, 'Video')

True

In [226]:
g.has_nodes(9, 'Video')

False

In [227]:
g.has_nodes(2, 'genre')

True

In [228]:
g.has_nodes(5, 'genre')

True

In [229]:
g.has_nodes(torch.tensor([1, 1, 2]), 'genre')

tensor([True, True, True])

In [230]:
g.has_nodes(torch.tensor([2, 3, 5]), 'Video')

tensor([True, True, True])

In [231]:
g.has_nodes(torch.tensor([7, 8, 9]), 'ID')

tensor([False, False, False])

In [232]:
g.has_nodes(torch.tensor([1, 8, 0]), 'ID')

tensor([ True, False,  True])

Return whether the graph contains the given edges.

In [233]:
g.has_edges_between(torch.tensor([1, 2]), torch.tensor([2, 3]),
                    ('ID', 'follower', 'ID'))

tensor([ True, False])

In [234]:
g.has_edges_between(torch.tensor([4, 5]), torch.tensor([8, 9]),
                     ('Video', 'belongs_to', 'genre'))

tensor([False, False])

In [235]:
g.has_edges_between(torch.tensor([3, 1]), torch.tensor([5, 4]),
                    ('ID', 'Cinema', 'Video'))

tensor([True, True])

Return the predecessor(s) of a particular node with the specified edge type.

In [236]:
#f = dgl.graph((torch.tensor([1, 1, 0, 3]), torch.tensor([2, 3, 3, 1])))
f.predecessors(1)

tensor([3])

In [237]:
g.predecessors(1, etype='follower')

tensor([0])

In [238]:
g.predecessors(4, etype='Cinema')

tensor([1])

In [239]:
g.predecessors(7, etype='belongs_to')

tensor([4])

Return the successor(s) of a particular node with the specified edge type.



In [240]:
f.successors(1)

tensor([2, 3])

In [241]:
g.successors(1, etype='follower')

tensor([2, 3])

In [242]:
g.successors(2, etype='Cinema')

tensor([6])

In [243]:
g.successors(4, etype='belongs_to')

tensor([7])

Return the edge ID(s) given the two endpoints of the edge(s).


Query for the edges.



In [244]:
f.edge_ids(1, 3)

1

In [245]:
f.edge_ids(torch.tensor([1, 3]), torch.tensor([3, 1]))

tensor([1, 3])

Get all edges for pairs of nodes.



In [246]:
f.edge_ids(torch.tensor([1, 3]), torch.tensor([3, 1]), return_uv=True)

(tensor([1, 3]), tensor([3, 1]), tensor([1, 3]))

If the graph has multiple edge types,

In [247]:
g.edge_ids(torch.tensor([1]), torch.tensor([2]), etype='follower')


tensor([1])

In [248]:
g.edge_ids(torch.tensor([5]), torch.tensor([8]), etype='belongs_to')

tensor([1])

Use a canonical edge type instead when there is ambiguity for an edge type.


In [249]:
g.edge_ids(torch.tensor([0, 1]), torch.tensor([1, 2]),
           etype=('ID', 'follower', 'ID'))

tensor([0, 1])

Return the source and destination node ID(s) given the edge ID(s).



In [250]:
f.find_edges(torch.tensor([1, 2]))

(tensor([1, 0]), tensor([3, 3]))

In [251]:
f.find_edges(torch.tensor([0, 3]))

(tensor([1, 3]), tensor([2, 1]))

In [252]:
g.find_edges(torch.tensor([1, 0]), 'Cinema')

(tensor([1, 3]), tensor([4, 5]))

In [253]:
g.find_edges(torch.tensor([1, 2]), 'belongs_to')

(tensor([5, 6]), tensor([8, 9]))

Return the incoming edges of the given nodes.



In [254]:
f.in_edges(torch.tensor([1, 0]))

(tensor([3]), tensor([1]))

In [255]:
f.in_edges(torch.tensor([1, 3]))

(tensor([3, 1, 0]), tensor([1, 3, 3]))

In [256]:
f.in_edges(torch.tensor([1, 2]), form='all')

(tensor([3, 1]), tensor([1, 2]), tensor([3, 0]))

In [257]:
g.in_edges(torch.tensor([0, 1]), etype='follower')

(tensor([2, 0]), tensor([0, 1]))

Return the outgoing edges of the given nodes.



In [258]:
f.out_edges(torch.tensor([2, 3]))

(tensor([3]), tensor([1]))

In [259]:
f.out_edges(torch.tensor([0, 2]))

(tensor([0]), tensor([3]))

In [260]:
f.out_edges(torch.tensor([1, 2]), form='all')

(tensor([1, 1]), tensor([2, 3]), tensor([0, 1]))

In [261]:
f.out_edges(torch.tensor([2, 3]), form='all')

(tensor([3]), tensor([1]), tensor([3]))

In [262]:
g.out_edges(torch.tensor([0, 2]), etype='follower')

(tensor([0, 2]), tensor([1, 0]))

In [263]:
g.out_edges(torch.tensor([0, 2]), etype='Cinema')

(tensor([2]), tensor([6]))

Return the in-degree(s) of the given nodes.



In [264]:
f.in_degrees()

tensor([0, 1, 1, 2])

In [265]:
f.in_degrees(torch.tensor([1, 2]))

tensor([1, 1])

In [266]:
g.in_degrees(torch.tensor([0, 3]), etype='follower')

tensor([1, 1])

In [267]:
g.in_degrees(torch.tensor([0, 3]), etype='Cinema')

tensor([0, 0])

In [268]:
g.in_degrees(torch.tensor([0, 6]), etype='Cinema')

tensor([0, 1])

Return the out-degree(s) of the given nodes.



In [269]:
f.out_degrees()

tensor([1, 2, 0, 1])

In [270]:
f.out_degrees(torch.tensor([1, 3]))

tensor([2, 1])

In [271]:
g.out_degrees(torch.tensor([1, 3]), etype='follower')

tensor([2, 0])

# Querying and manipulating node/edge ID type

The data type for storing the structure-related graph information such as node and edge IDs.



In [272]:
src_ids = torch.tensor([1, 1, 2, 3])
dst_ids = torch.tensor([1, 2, 3, 2])
g = dgl.graph((src_ids, dst_ids))
g.idtype

torch.int64

In [273]:
g = dgl.graph((src_ids, dst_ids), idtype=torch.int32)
g.idtype

torch.int32

Cast the graph to one with idtype int64



In [274]:
f = dgl.graph((torch.tensor([1, 1, 0, 3]).int(), torch.tensor([2, 3, 3, 1]).int()))

f.ndata['blue'] = torch.ones(4, 2)
f.idtype

torch.int32

In [275]:
f_long = f.long()
f_long.idtype

torch.int64

In [276]:
f.idtype

torch.int32

In [277]:
f_long.edges()

(tensor([1, 1, 0, 3]), tensor([2, 3, 3, 1]))

In [278]:
f_long.ndata

{'blue': tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])}

Cast the graph to one with idtype int32



In [279]:
f = dgl.graph((torch.tensor([1, 1, 0, 3]), torch.tensor([2, 3, 3, 1])))
f.ndata['Blue'] = torch.ones(4, 3)
f.idtype

torch.int64

In [280]:
f_int = f.int()
f_int.idtype

torch.int32

In [281]:
f.idtype

torch.int64

In [282]:
f_int.edges()

(tensor([1, 1, 0, 3], dtype=torch.int32),
 tensor([2, 3, 3, 1], dtype=torch.int32))

In [283]:
f_int.ndata

{'Blue': tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])}

# Using Node/edge features


In [284]:
g = dgl.heterograph({
    ('ID', 'follower', 'ID'): (torch.tensor([0, 1, 1, 2]), torch.tensor([1, 2, 3, 0])),
    ('ID', 'Cinema', 'Video'): (torch.tensor([3, 1, 2]), torch.tensor([5, 4, 6])),
    ('Video', 'belongs_to', 'genre'): (torch.tensor([4, 5, 6]), torch.tensor([7, 8, 9]))
})


Return a node view



In [285]:
f.nodes()

tensor([0, 1, 2, 3])

In [286]:
g.nodes('ID')

tensor([0, 1, 2, 3])

In [287]:
g.nodes('Video')

tensor([0, 1, 2, 3, 4, 5, 6])

In [288]:
g.nodes['Video'].data['h'] = torch.ones(7, 1)
g.nodes['Video'].data['h']

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

Return a node data view for setting/getting node features



In [289]:
f.ndata['h'] = torch.ones(4, 1)
f.ndata['h']

tensor([[1.],
        [1.],
        [1.],
        [1.]])

In [290]:
g.ndata['h'] = {'ID': torch.zeros(2, 1), 'ID': torch.ones(4, 1)}
g.ndata['h']

{'ID': tensor([[1.],
         [1.],
         [1.],
         [1.]]),
 'Video': tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]])}

In [291]:
g.ndata['h'] = {'Video': torch.ones(7, 1)}
g.ndata['h']

{'ID': tensor([[1.],
         [1.],
         [1.],
         [1.]]),
 'Video': tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]])}

Return an edge view



In [292]:
f.edges()

(tensor([1, 1, 0, 3]), tensor([2, 3, 3, 1]))

In [293]:
f.edges(form='all', order='srcdst')

(tensor([0, 1, 1, 3]), tensor([3, 2, 3, 1]), tensor([2, 0, 1, 3]))

In [294]:
g.edges['follower'].data['h'] = torch.ones(4, 1)
g.edges['follower'].data['h']

tensor([[1.],
        [1.],
        [1.],
        [1.]])

Return an edge data view for setting/getting edge features.



In [295]:
f.edata['h'] = torch.ones(4, 1)
f.edata['h']

tensor([[1.],
        [1.],
        [1.],
        [1.]])

In [296]:
g.edata['h'] = {('ID', 'follower', 'ID'): torch.zeros(4, 1),
                ('ID', 'Cinema', 'Video'): torch.ones(3, 1)}
g.edata['h']

{('ID',
  'Cinema',
  'Video'): tensor([[1.],
         [1.],
         [1.]]),
 ('ID',
  'follower',
  'ID'): tensor([[0.],
         [0.],
         [0.],
         [0.]])}

In [297]:
g.edata['h'] = {('Video', 'belongs_to', 'genre'): torch.ones(3, 1)}
g.edata['h']

{('ID',
  'Cinema',
  'Video'): tensor([[1.],
         [1.],
         [1.]]),
 ('ID',
  'follower',
  'ID'): tensor([[0.],
         [0.],
         [0.],
         [0.]]),
 ('Video',
  'belongs_to',
  'genre'): tensor([[1.],
         [1.],
         [1.]])}

Return the node feature schemes for the specified type.



In [298]:
f.ndata['h1'] = torch.randn(4, 1)
f.ndata['h2'] = torch.randn(4, 2)
f.node_attr_schemes()

{'Blue': Scheme(shape=(3,), dtype=torch.float32),
 'h': Scheme(shape=(1,), dtype=torch.float32),
 'h1': Scheme(shape=(1,), dtype=torch.float32),
 'h2': Scheme(shape=(2,), dtype=torch.float32)}

In [299]:
g = dgl.heterograph({('ID', 'follower', 'ID'):
                     (torch.tensor([1, 2]), torch.tensor([3, 4]))})
g.nodes['ID'].data['h1'] = torch.randn(5, 1)
g.nodes['ID'].data['h2'] = torch.randn(5, 2)
g.node_attr_schemes('ID')

{'h1': Scheme(shape=(1,), dtype=torch.float32),
 'h2': Scheme(shape=(2,), dtype=torch.float32)}

Return the edge feature schemes for the specified type.



In [300]:
f.edata['h1'] = torch.randn(4, 1)
f.edata['h2'] = torch.randn(4, 2)
f.edge_attr_schemes()

{'h': Scheme(shape=(1,), dtype=torch.float32),
 'h1': Scheme(shape=(1,), dtype=torch.float32),
 'h2': Scheme(shape=(2,), dtype=torch.float32)}

In [301]:
g.edges['follower'].data['h1'] = torch.randn(2, 1)
g.edges['follower'].data['h2'] = torch.randn(2, 2)
g.edge_attr_schemes('follower')

{'h1': Scheme(shape=(1,), dtype=torch.float32),
 'h2': Scheme(shape=(2,), dtype=torch.float32)}

Return a node view for source nodes



In [302]:
b.srcnodes('ID')

tensor([0])

In [303]:
b.srcnodes('Video')

tensor([0, 1])

In [304]:
b.srcnodes['ID'].data['h'] = torch.ones(1, 1)
b.srcnodes['ID'].data['h']

tensor([[1.]])

In [305]:
n.srcnodes('ID')

tensor([0, 1])

In [306]:
n.srcnodes['genre'].data['h'] = torch.ones(3, 1)
n.srcnodes['genre'].data['h']

tensor([[1.],
        [1.],
        [1.]])

Return a node view for destination nodes



In [307]:
b.dstnodes('genre')

tensor([0, 1, 2])

In [308]:
b.dstnodes['genre'].data['h'] = torch.ones(3, 1)
b.dstnodes['genre'].data['h']

tensor([[1.],
        [1.],
        [1.]])

In [309]:
n.dstnodes('Video')

tensor([0, 1])

In [310]:
n.dstnodes['Video'].data['h'] = torch.ones(2, 1)
n.dstnodes['Video'].data['h']

tensor([[1.],
        [1.]])

Return a node data view for setting/getting source node features.



In [311]:
f.srcdata['h'] = torch.ones(4, 1)
f.srcdata['h']

tensor([[1.],
        [1.],
        [1.],
        [1.]])

In [312]:
g = dgl.heterograph({
    ('ID', 'follower', 'ID'): (torch.tensor([0, 1, 1, 2]), torch.tensor([1, 2, 3, 0])),
    ('ID', 'Cinema', 'Video'): (torch.tensor([3, 1, 2]), torch.tensor([5, 4, 6])),
    ('Video', 'belongs_to', 'genre'): (torch.tensor([4, 5, 6]), torch.tensor([7, 8, 9]))
})


In [313]:
g.srcdata['h'] = {'ID': torch.zeros(4, 1), 'Video': torch.ones(7, 1)}
g.srcdata['h']

{'ID': tensor([[0.],
         [0.],
         [0.],
         [0.]]),
 'Video': tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]])}

Return a node data view for setting/getting destination node features.



In [314]:
f.dstdata['h'] = torch.ones(4, 1)
f.dstdata['h']

tensor([[1.],
        [1.],
        [1.],
        [1.]])

In [315]:
g.dstdata['h'] = {'ID': torch.zeros(4, 1), 'genre': torch.ones(10, 1)}
g.dstdata['h']

{'ID': tensor([[0.],
         [0.],
         [0.],
         [0.]]),
 'Video': tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]),
 'genre': tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]])}

In [316]:
g.dstdata['h'] = {'genre': torch.ones(10, 1)}
g.dstdata['h']

{'ID': tensor([[0.],
         [0.],
         [0.],
         [0.]]),
 'Video': tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]),
 'genre': tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]])}

# Transforming graph

Alias of dgl.node_subgraph()

In [317]:
g.subgraph

<bound method node_subgraph of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.edge_subgraph()

In [318]:
g.edge_subgraph

<bound method edge_subgraph of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.node_type_subgraph().



In [319]:
g.node_type_subgraph

<bound method node_type_subgraph of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.edge_type_subgraph().



In [320]:
g.edge_type_subgraph

<bound method edge_type_subgraph of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Return the relation slice of this graph.

In [321]:
g.__getitem__

<bound method DGLGraph.__getitem__ of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.line_graph()

In [322]:
g.line_graph

<bound method line_graph of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.reverse()

In [323]:
g.reverse

<bound method reverse of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.add_self_loop()

In [324]:
g.add_self_loop

<bound method add_self_loop of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.remove_self_loop()

In [325]:
g.remove_self_loop

<bound method remove_self_loop of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.to_simple()

In [326]:
g.to_simple

<bound method to_simple of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Convert a DGL graph to a cugraph.Graph and return

In [327]:
g.to_cugraph

<bound method to_cugraph of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

Alias of dgl.reorder_graph()

In [328]:
g.reorder_graph

<bound method reorder_graph of Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 3, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 3},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])>

# Subgraph Extraction Ops

Return a subgraph induced on the given nodes.



In [329]:
sf = dgl.node_subgraph(f, [0, 1, 1])
sf

Graph(num_nodes=3, num_edges=0,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [330]:
sf.edges()

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [331]:
sf.ndata[dgl.NID]

tensor([0, 1, 1])

In [332]:
sf.edata[dgl.EID]

tensor([], dtype=torch.int64)

In [333]:
nodesf = torch.tensor([True, True, False, False])
dgl.node_subgraph(f, nodesf)

Graph(num_nodes=2, num_edges=0,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [334]:
f.ndata['x'] = torch.arange(8).view(4, 2)
sf = dgl.node_subgraph(f, [0, 1, 1])
sf

Graph(num_nodes=3, num_edges=0,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [335]:
sg = dgl.node_subgraph(g, {'ID': [1, 1]})
sg

Graph(num_nodes={'ID': 2, 'Video': 0, 'genre': 0},
      num_edges={('ID', 'Cinema', 'Video'): 0, ('ID', 'follower', 'ID'): 0, ('Video', 'belongs_to', 'genre'): 0},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])

Return a subgraph induced on the given edges.

In [336]:
sf = dgl.edge_subgraph(f, [0, 1])
sf

Graph(num_nodes=3, num_edges=2,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [337]:
sf.edges()

(tensor([0, 0]), tensor([1, 2]))

In [338]:
sf.ndata[dgl.NID]

tensor([1, 2, 3])

In [339]:
sf.edata[dgl.EID]

tensor([0, 1])

In [340]:
nodesf = torch.tensor([True, False, False, False])
dgl.edge_subgraph(f, nodesf)

Graph(num_nodes=2, num_edges=1,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [341]:
f.ndata['x'] = torch.arange(8).view(4, 2)
sf = dgl.edge_subgraph(f, [1, 1])
sf

Graph(num_nodes=2, num_edges=2,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [342]:
sf.ndata[dgl.NID]

tensor([1, 3])

In [343]:
sf.ndata['x']

tensor([[2, 3],
        [6, 7]])

In [344]:
sg = dgl.edge_subgraph(g, {('ID', 'follower', 'ID'): [0],
                           ('ID', 'Cinema', 'Video'): [1, 2]
                           })
print(sg)

Graph(num_nodes={'ID': 3, 'Video': 2, 'genre': 0},
      num_edges={('ID', 'Cinema', 'Video'): 2, ('ID', 'follower', 'ID'): 1, ('Video', 'belongs_to', 'genre'): 0},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])


Return the subgraph induced on given node types.

In [345]:
g.nodes['ID'].data['h'] = torch.tensor([[0.], [1.], [4.], [3.]])

In [346]:
sub_g = g.node_type_subgraph(['ID'])
print(sub_g)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32)}
      edata_schemes={})


In [347]:
sub_g.nodes['ID'].data['h']

tensor([[0.],
        [1.],
        [4.],
        [3.]])

Return the subgraph induced on given edge types

In [348]:
g.edges['follower'].data['h'] = torch.tensor([[0.], [1.], [4.], [3.]])

In [349]:
sub_g = g.edge_type_subgraph(['follower'])
sub_g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32)})

In [350]:
sub_g.edges['follower'].data['h']

tensor([[0.],
        [1.],
        [4.],
        [3.]])

Return the subgraph induced on the inbound edges of all the edge types of the given nodes.



In [351]:
f.edata['w'] = torch.arange(8).view(4, 2)

In [352]:
sf = dgl.in_subgraph(f, [2, 0])
sf

Graph(num_nodes=4, num_edges=1,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'w': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [353]:
sf.edges()

(tensor([1]), tensor([2]))

In [354]:
sf.edata[dgl.EID]

tensor([0])

In [355]:
sf.edata['w']

tensor([[0, 1]])

In [356]:
sf = dgl.in_subgraph(f, [2, 0], relabel_nodes=True)
sf

Graph(num_nodes=2, num_edges=1,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'w': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [357]:
sf.edges()

(tensor([0]), tensor([1]))

In [358]:
sf.edata[dgl.EID]

tensor([0])

In [359]:
sf.ndata[dgl.NID]

tensor([1, 2])

In [360]:
sub_g = g.in_subgraph({'ID': [0], 'genre': [2]})
sub_g

Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 0, ('ID', 'follower', 'ID'): 1, ('Video', 'belongs_to', 'genre'): 0},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])

Return the subgraph induced on the outbound edges of all the edge types of the given nodes.



In [361]:
f.edata['w'] = torch.arange(8).view(4, 2)
sf = dgl.out_subgraph(f, [2, 0])
sf

Graph(num_nodes=4, num_edges=1,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'w': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [362]:
sf.edges()

(tensor([0]), tensor([3]))

In [363]:
sf.edata[dgl.EID]

tensor([2])

In [364]:
sf.edata['w']

tensor([[4, 5]])

In [365]:
sf = dgl.out_subgraph(f, [2, 0], relabel_nodes=True)
sf

Graph(num_nodes=2, num_edges=1,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'w': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [366]:
sf = dgl.out_subgraph(f, [2, 0], relabel_nodes=True)
sf

Graph(num_nodes=2, num_edges=1,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'w': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [367]:
sf.edges()

(tensor([0]), tensor([1]))

In [368]:
sf.edata[dgl.EID]

tensor([2])

In [369]:
sf.ndata[dgl.NID]

tensor([0, 3])

In [370]:
sg = g.out_subgraph({'ID': [1]})
sg

Graph(num_nodes={'ID': 4, 'Video': 7, 'genre': 10},
      num_edges={('ID', 'Cinema', 'Video'): 1, ('ID', 'follower', 'ID'): 2, ('Video', 'belongs_to', 'genre'): 0},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])

Return the subgraph induced by k-hop in-neighborhood of the specified node(s)

In [371]:
f.edata['w'] = torch.arange(8).view(4, 2)
sf, inverse_indices = dgl.khop_in_subgraph(f, 0, k=2)
sf

Graph(num_nodes=1, num_edges=0,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'w': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [372]:
sf.edges()

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [373]:
sf.edata[dgl.EID]  # original edge IDs

tensor([], dtype=torch.int64)

In [374]:
sf.edata['w']  # also extract the features

tensor([], size=(0, 2), dtype=torch.int64)

In [375]:
inverse_indices

tensor([0])

In [376]:
sg, inverse_indices = dgl.khop_in_subgraph(g, {'ID': 0}, k=2)
sg

Graph(num_nodes={'ID': 3, 'Video': 0, 'genre': 0},
      num_edges={('ID', 'Cinema', 'Video'): 0, ('ID', 'follower', 'ID'): 3, ('Video', 'belongs_to', 'genre'): 0},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])

Return the subgraph induced by k-hop out-neighborhood of the specified node(s)

In [377]:
f.edata['w'] = torch.arange(8).view(4, 2)
sf, inverse_indices = dgl.khop_out_subgraph(f, 0, k=2)
sf

Graph(num_nodes=3, num_edges=3,
      ndata_schemes={'Blue': Scheme(shape=(3,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'x': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32), 'h1': Scheme(shape=(1,), dtype=torch.float32), 'h2': Scheme(shape=(2,), dtype=torch.float32), 'w': Scheme(shape=(2,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [378]:
sf.edges()

(tensor([0, 1, 2]), tensor([2, 2, 1]))

In [379]:
sf.edata[dgl.EID]

tensor([2, 1, 3])

In [380]:
sf.edata['w']

tensor([[4, 5],
        [2, 3],
        [6, 7]])

In [381]:
inverse_indices

tensor([0])

In [382]:
sg, inverse_indices = dgl.khop_out_subgraph(g, {'ID': 0}, k=2)
sg

Graph(num_nodes={'ID': 4, 'Video': 1, 'genre': 0},
      num_edges={('ID', 'Cinema', 'Video'): 1, ('ID', 'follower', 'ID'): 4, ('Video', 'belongs_to', 'genre'): 0},
      metagraph=[('ID', 'Video', 'Cinema'), ('ID', 'ID', 'follower'), ('Video', 'genre', 'belongs_to')])

# Save & Load

In [383]:
from dgl.data.utils import save_graphs
graph_labels = {"glabel": torch.tensor([0, 1, 2, 3])}
save_graphs("./data.bin", [g, f, b, n], graph_labels)

In [384]:
from dgl.data.utils import load_graphs
glist, label_dict = load_graphs("./data.bin")
glist, label_dict = load_graphs("./data.bin", [0])